# Импорт библиотек

In [1]:
import numpy as np
from gzip import open # NB: overrides standard open()
import pickle as pkl
import pandas as pd
import warnings 
from glob import glob
import os
import math
import functools

### Отключим предупреждения Anaconda

In [2]:
warnings.simplefilter('ignore')

# Загрузим данные

In [3]:
Xdata_numpy = pkl.load(open('C:/Users/admin/Desktop/PLS_continue_2020/X.pkl.gz', 'rb'))
Ydata = pkl.load(open('C:/Users/admin/Desktop/PLS_continue_2020/y.pkl.gz', 'rb'))

# N-PLS1

## Выберем столбец у с которым будем работать

In [4]:
m=0

# Приближение к правильному алгоритму

In [104]:
from sklearn.model_selection import train_test_split

In [219]:
X_train, X_test, y_train, y_test = train_test_split(
    Xdata_numpy["X"], Ydata.iloc[:,m], test_size=0.2857 
    )

In [220]:
X_train.shape[0]

25

In [355]:
x=X_train

In [356]:
y=y_train

In [357]:
y=np.array(y)

In [358]:
x=np.array(x)

In [359]:
yo=y

In [345]:
# a-numpy array;  b-pandas array
def error(a,b):
    s=0
    for i in range(0,len(a)):
        s+=((a[i]-b.iloc[i]))**2
    return s

In [368]:
parameters={'numbers_of_learning':3, 'numbers_of_components':15}

In [371]:

for v in range(0,parameters['numbers_of_learning']):
    X_train, X_test, y_train, y_test = train_test_split(
        Xdata_numpy["X"], Ydata.iloc[:,m], test_size=0.2857 
        )
    x=X_train
    y=y_train
    y=np.array(y)
    x=np.array(x)

    Tt=[]
    mass=np.zeros([y.shape[0]])
    for f in range(0,parameters['numbers_of_components']):
        z=np.zeros([x.shape[1],x.shape[2]])
        for i in range(0,x.shape[2]):
            for j in range(0,x.shape[1]):
                zsum=0
                for k in range(0,x.shape[0]):
                    zsum= zsum+ x[k,j,i]*y[k]
                z[j,i]=zsum
        Wk, S, WI = np.linalg.svd(z)
        w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
        w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
        
        t=[]
        for h in range(0,x.shape[0]):
            t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
        t=np.array(t).reshape(x.shape[0],1)
        Tt+=[t]
        T=np.array(Tt).reshape(x.shape[0],f+1)
        bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
        for g in range(0,x.shape[0]):
            x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
        y=y-(np.dot(T,bf))
        mass+=np.dot(T,bf)
        print(error(mass,y_train))
        output[f]+=error(mass,y_train)
    print()
np.array(output).reshape(1,parameters['numbers_of_components'])

5415.247299074339
5413.790454451667
4728.659885673288
3732.608581075338
3520.28297221468
2650.0034985129414
1747.361353630304
1043.595264546046
556.7091328113486
312.1828928350037
209.52674203418636
162.85448645906126
69.87062893415282
23.331300388471465
12.280072130011904

5465.209838291314
5294.7777412017585
4978.758259902588
4477.876446640462
4098.808330110305
3437.106112069044
2973.4810551091755
2567.966734345999
2092.969605595385
1687.2039450850796
1254.787685168586
1132.2054115278925
585.704348544216
240.63691735488467
72.6684294637976

5450.194199964582
4930.333422227027
4576.119806534856
4300.815696356478
3855.7601773270567
3140.433927125711
2660.8926247040317
2463.170244788937
2171.271998057505
1292.9768188113112
619.8254949222896
309.3566549791087
137.82678772795504
61.68889268562566
53.60363593291268



array([[5450.19419996, 4930.33342223, 4576.11980653, 4300.81569636,
        3855.76017733, 3140.43392713, 2660.8926247 , 2463.17024479,
        2171.27199806, 1292.97681881,  619.82549492,  309.35665498,
         137.82678773,   61.68889269,   53.60363593]])

In [372]:
output

array([5450.19419996, 4930.33342223, 4576.11980653, 4300.81569636,
       3855.76017733, 3140.43392713, 2660.8926247 , 2463.17024479,
       2171.27199806, 1292.97681881,  619.82549492,  309.35665498,
        137.82678773,   61.68889269,   53.60363593])

###### Массив mass собирает разложение обратно, и именно он важен для определения числа комронент

In [339]:
Tt=[]
mass=np.zeros([y.shape[0]])
for f in range(0,55):
    z=np.zeros([x.shape[1],x.shape[2]])
    for i in range(0,x.shape[2]):
        for j in range(0,x.shape[1]):
            zsum=0
            for k in range(0,x.shape[0]):
                zsum= zsum+ x[k,j,i]*y[k]
            z[j,i]=zsum
    Wk, S, WI = np.linalg.svd(z)
    w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
    w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
    
    t=[]
    for h in range(0,x.shape[0]):
        t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
    t=np.array(t).reshape(x.shape[0],1)
    Tt+=[t]
    T=np.array(Tt).reshape(x.shape[0],f+1)
    bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
    for g in range(0,x.shape[0]):
        x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
    y=y-(np.dot(T,bf))
    mass+=np.dot(T,bf)
    print(error(mass,y_train))

3.2573949359354034e+37
1.3736859842840645e+38
1.8175512898144767e+38
2.422464079860449e+38
2.3984639952295402e+38
2.6797800601292366e+38
3.1934498320857485e+38
3.1873893837616863e+38
3.2013521653352052e+38
3.167160875903498e+38
2.6311524878397894e+38
2.7470024619541774e+38
2.8539096693206566e+38
3.1938798296302585e+38
3.161113130762126e+38
3.418633318587955e+38
3.458668894413968e+38
3.499149693556257e+38
3.590370932664191e+38
3.5951144823316104e+38
3.6040156866470576e+38
3.6172311677698454e+38
3.619996016509866e+38
3.623215991294016e+38
3.62318830697043e+38
3.623014976555737e+38
3.623053086161273e+38
3.6229212222125596e+38
3.62340241706419e+38
3.6239120187036645e+38
3.619415220095947e+38
3.624720501107461e+38
3.6251621895657074e+38
3.6044685296632485e+38
3.640420922509956e+38
3.6498354992454605e+38
3.840705311084655e+38
6.805779930052282e+38
9.567898050035575e+38
8.336496258388958e+40
7.716666344822102e+42
2.418333329584541e+45
3.372777135112628e+47
1.5131326699362803e+48
2.51089528852

# Попробую оформить всё в виде полноценного класса

In [ ]:
class N_PLS1():
    def error(a,b):
        s=0
        for i in range(0,len(a)):
            s+=((a[i]-b.iloc[i]))**2
        return s
    def function_1(x,y,fn):
        Tt=[]
        mass=np.zeros([y.shape[0]])
        output=[]
        for f in range(0,fn):
            z=np.zeros([x.shape[1],x.shape[2]])
            for i in range(0,x.shape[2]):
                for j in range(0,x.shape[1]):
                    zsum=0
                    for k in range(0,x.shape[0]):
                        zsum= zsum+ x[k,j,i]*y[k]
                    z[j,i]=zsum
            Wk, S, WI = np.linalg.svd(z)
            w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
            w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
            t=[]
            for h in range(0,x.shape[0]):
                t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
            t=np.array(t).reshape(x.shape[0],1)
            Tt+=[t]
            T=np.array(Tt).reshape(x.shape[0],f+1)
            bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
            for g in range(0,x.shape[0]):
                x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
            y=y-(np.dot(T,bf))
            mass+=np.dot(T,bf)
            output+=[error(mass,y_train)]
        return output
    def fit(x,y,fn):
        
            
        return y

In [179]:
y

array([34.19308759, 35.72443085, 46.32211944, 35.69579348, 47.44184036,
       33.34204937, 42.44462856, 39.9018207 , 44.39973652, 54.64066407,
       53.71596802, 44.04933373, 36.11255278, 34.24246421, 36.22622795,
       26.1974747 , 41.0398005 , 31.41066637, 34.07047331, 42.14770231,
       45.44679116, 21.41118209, 44.54683093, 35.35460545, 36.47993911])

In [54]:
print(y.shape)

()


In [290]:
function_1(x,y,25)

[41286.08749854996,
 38302.110243945746,
 20444.93670747345,
 13639.701185793923,
 12439.263695246093,
 10789.974224403275,
 8598.56556560382,
 7968.967506273657,
 7064.892691449656,
 6299.901986225843,
 5402.479038865533,
 3654.37066066749,
 3213.2931840440165,
 2682.495895172844,
 1872.8829464517519,
 1219.8609339979532,
 901.9277716135091,
 553.7388536351104,
 252.14526974505077,
 118.10074288670174,
 38.94981464085392,
 21.429430366938657,
 5.790438269180314,
 2.0139509572786625,
 0.8471606395169777]

In [331]:
function_2(Xdata_numpy["X"], Ydata.iloc[:,m], 1, 29, 0.2857)

[5881.081773936589, 5195.194135948176, 4595.753850448796, 3697.4301685787964, 2393.408183981792, 1730.2577746392737, 1137.495276542045, 836.8981056708691, 502.7815170755121, 414.6809716536083, 340.22588862523133, 220.3599620901575, 133.94619309571854, 80.35650253454206, 36.68023612039321, 30.981077033990275, 12.515987829782226, 4.3311267341245285, 1.3863370980748204, 0.7705193007573259, 0.24346041599041418, 0.10774205447739513, 0.016419091188545464, 0.004574090183078223, 0.002686760816077912, 0.0029362552828107406, 0.001584657029826227, 0.002247869747972181, 0.001431492737689517]


[[6389.803909882158,
  5916.599983237266,
  4999.201390296729,
  3737.9558974357187,
  3166.641003683795,
  2791.2273526995987,
  2561.347949798782,
  2155.46758995359,
  1849.5508307278424,
  763.6974357205736,
  628.0873793772564,
  342.44850957395926,
  121.53125255132385,
  46.6384235185164,
  21.163442969920084,
  12.74031224792265,
  10.285472319360716,
  10.064138308163399,
  8.805455535457378,
  8.441746471802308,
  5.545303363744137,
  5.35881305379689,
  1.399291007825839,
  0.03968376890055237,
  0.0006888542845988004,
  9.647630520136338e-09,
  8.177593269649294e-08,
  2.221234637007994e-08,
  8.217963141638592e-06]]

In [ ]:
nbn